In [1]:
import pandas as pd 
import numpy as np
from qiime2 import Artifact
import os

Import of the final sample dataset obtained in the EDA notebook: 

In [2]:
data = pd.read_csv('data_final_groups.csv',dtype = {"sample_name": "str"},index_col=0)
data

,sample_name,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_frequency,...,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency,group
0,10317.000023139,No,No,No,No,60s,69.0,69.0,Yes,Rarely (a few times/month),...,Regularly (3-5 times/week),Daily,Never,NaN,Remained stable,63,kilograms,Regularly (3-5 times/week),Regularly (3-5 times/week),Treatment
1,10317.000027811,No,No,No,No,40s,46.0,46.0,Yes,Rarely (a few times/month),...,Rarely (a few times/month),Regularly (3-5 times/week),Rarely (a few times/month),NaN,Remained stable,81,kilograms,Occasionally (1-2 times/week),Rarely (less than once/week),Treatment
2,10317.000031513,Self-diagnosed,No,No,Not provided,30s,37.0,37.0,Yes,Regularly (3-5 times/week),...,Never,Rarely (a few times/month),Rarely (a few times/month),NaN,Remained stable,49,kilograms,Occasionally (1-2 times/week),Rarely (less than once/week),Treatment
3,10317.000033280,No,No,No,No,60s,60.0,60.0,Yes,Regularly (3-5 times/week),...,Never,Never,Never,NaN,Remained stable,68,kilograms,Occasionally (1-2 times/week),Regularly (3-5 times/week),Treatment
4,10317.000038261,No,No,No,No,40s,48.0,48.0,Yes,Regularly (3-5 times/week),...,Never,Never,Occasionally (1-2 times/week),NaN,Remained stable,70,kilograms,Rarely (less than once/week),Occasionally (1-2 times/week),Treatment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,10317.000065667,No,No,No,No,20s,22.0,22.0,Yes,Occasionally (1-2 times/week),...,Never,Never,Rarely (a few times/month),NaN,Remained stable,61,kilograms,Regularly (3-5 times/week),Never,Control
376,10317.000065673,No,No,Yes,No,20s,20.0,20.0,No,Never,...,Never,Never,Occasionally (1-2 times/week),NaN,Remained stable,79,kilograms,Never,Rarely (less than once/week),Control
377,10317.000065675,No,No,No,No,20s,21.0,21.0,No,Never,...,Never,Never,Daily,NaN,Remained stable,51,kilograms,Never,Regularly (3-5 times/week),Control
378,10317.000065681,No,No,Yes,No,20s,21.0,21.0,No,Never,...,Never,Never,Rarely (a few times/month),NaN,Remained stable,137,kilograms,Rarely (less than once/week),Rarely (less than once/week),Control


In [3]:
data.specialized_diet_exclude_refined_sugars.value_counts()

Yes    190
No     190
Name: specialized_diet_exclude_refined_sugars, dtype: int64

## Metadata file creation

In [4]:
metadata = data.sample_name.to_frame()

In [5]:
metadata = metadata.rename(columns={'sample_name': 'sample-id'})

In [6]:
# Aggiungiamo al manifest le informazioni riguardo l'età, il sesso e l'uso degli antibiotici
metadata['sex'] = data['sex']
metadata['age_cat'] = data['age_cat']
metadata['refined_sugars'] = data['specialized_diet_exclude_refined_sugars']
metadata

,sample-id,sex,age_cat,refined_sugars
0,10317.000023139,female,60s,Yes
1,10317.000027811,male,40s,Yes
2,10317.000031513,female,30s,Yes
3,10317.000033280,female,60s,Yes
4,10317.000038261,female,40s,Yes
...,...,...,...,...
375,10317.000065667,female,20s,No
376,10317.000065673,female,20s,No
377,10317.000065675,female,20s,No
378,10317.000065681,female,20s,No


In [7]:
metadata['sample-id'].to_csv("ids.txt",header=None,index=None)

In [8]:
metadata.refined_sugars.value_counts()

Yes    190
No     190
Name: refined_sugars, dtype: int64

In [9]:
metadata.to_csv('sample_metadata.tsv', sep='\t', index=False)

In [10]:
from qiime2 import Metadata
sample_metadata = Metadata.load(os.getcwd()+'/sample_metadata.tsv')

Import Fasta 99% otus containing Taxonomy

In [11]:
!qiime tools import --type 'FeatureData[Sequence]' --input-path gg_13_8_otus/rep_set/99_otus.fasta \
    --output-path rep_set.qza

Imported gg_13_8_otus/rep_set/99_otus.fasta as DNASequencesDirectoryFormat to rep_set.qza


In [19]:
otu_table_df = pd.read_csv("otu_table.tsv",sep="\t", skiprows=1)

Aggiungere comando per tirare giù otutable

In [15]:
!export ctx=Pick_closed-reference_OTUs-Greengenes-Illumina-16S-V4-5c6506

In [16]:
!redbiom fetch samples --from ids.txt --context $ctx --output ids_extended.biom

Traceback (most recent call last):
  File "/home/LABORATORI/gr887277/.local/bin/redbiom", line 5, in <module>
    from redbiom.commands import cli
ModuleNotFoundError: No module named 'redbiom'


Aggiungere comando per trasformare .biom in .tsv

In [ ]:
!biom convert -i ids.biom -o otu_table.tsv --to-tsv 

In [20]:
otu_table_df

,#OTU ID,10317.000023139.25853,10317.000027811.25853,10317.000031513.25853,10317.000033280.31503,10317.000038261.33800,10317.000041075.48742,10317.000050189.26531,10317.000051222.53379,10317.000052017.26531,...,10317.000062080.26531,10317.000062200.26480,10317.000062201.69016,10317.000063259.31503,10317.000065483.33800,10317.000065667.26478,10317.000065673.25853,10317.000065675.25853,10317.000065681.25853,10317.000065684.25853
0,305247,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4324196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1107674,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,175836,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,21.0
4,3410999,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6899,189760,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,19.0,1.0
6900,199962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6901,4425390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6902,4377144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
!biom convert -i otu_table_filt.tsv -o otu_table_filt.biom --table-type="OTU table" --to-hdf5

In [21]:
!qiime feature-table filter-seqs --i-data rep_set.qza --m-metadata-file otu_table.tsv --o-filtered-data rep_set_filt.qza

Saved FeatureData[Sequence] to: rep_set_filt.qza


In [135]:
!biom convert -i otu_table_reorder.tsv -o otu_table_reorder.biom --table-type="OTU table" --to-hdf5

In [22]:
!qiime tools import \
  --input-path otu_table.biom\
  --type 'FeatureTable[Frequency]' \
  --input-format BIOMV210Format \
  --output-path feature_table.qza

Imported otu_table.biom as BIOMV210Format to feature_table.qza


In [23]:
from qiime2 import Artifact
otu_table = Artifact.load("feature_table.qza")
rep_set = Artifact.load("rep_set_filt.qza")

In [24]:
ot_df = otu_table.view(pd.DataFrame) 

In [25]:
ot_df

,2941399,4425663,353737,363646,1090458,922239,845273,185021,293665,4453060,...,213762,179905,4324040,333726,199147,1808779,173916,435264,325586,4307347
10317.000023139.25853,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000027811.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,30.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000031513.25853,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000033280.31503,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000038261.33800,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317.000065667.26478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000065673.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000065675.25853,0.0,81.0,0.0,0.0,92.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000065681.25853,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [112]:
ot_df_filt = ot_df.loc[ot_df.index.isin(meta_table_filt["sample-id"])]

In [115]:
ot_df_filt.to_csv("otu_table_filt.tsv",sep="\t")

In [26]:
from qiime2.plugins import alignment, phylogeny
mafft_alignment = alignment.methods.mafft(rep_set)
masked_mafft_alignment = alignment.methods.mask(mafft_alignment.alignment)
unrooted_tree = phylogeny.methods.fasttree(masked_mafft_alignment.masked_alignment)
rooted_tree = phylogeny.methods.midpoint_root(unrooted_tree.tree)


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 1 /tmp/qiime2/gr887277/data/31ce2045-ca0e-491c-8f36-3162bde21e4b/data/dna-sequences.fasta



inputfile = orig
6904 x 1951 - 1299 d
nthread = 1
nthreadpair = 1
nthreadtb = 1
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..

There are 2547 ambiguous characters.
 6901 / 6904 (thread    0)
done.

Constructing a UPGMA tree (efffree=0) ... 
 6900 / 6904
done.

Progressive alignment 1/2... 
STEP  4701 / 6903 (thread    0)
Reallocating..done. *alloclen = 4957
STEP  6701 / 6903 (thread    0)
Reallocating..done. *alloclen = 7181
STEP  6801 / 6903 (thread    0)
Reallocating..done. *alloclen = 8406

Reallocating..done. *alloclen = 10312
STEP  6901 / 6903 (thread    0) h
done.

Making a distance matrix from msa.. 
 6900 / 6904 (thread    0)
done.

Constructing a UPGMA tree (efffree=1) ... 
 6900 / 6904
done.

Progressive alignment 2/2... 
STEP  4601 / 6903 (thread    0)
Reallocating..done. *alloclen = 4953
STEP  6701 / 6903 (thread    0)
Reallocating..done. *alloclen = 5975
STE

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /tmp/qiime2/gr887277/data/72001f7a-e1e9-4454-bcb0-7727ad60d18f/data/aligned-dna-sequences.fasta



FastTree Version 2.1.11 Double precision (No SSE3)
Alignment: /tmp/qiime2/gr887277/data/72001f7a-e1e9-4454-bcb0-7727ad60d18f/data/aligned-dna-sequences.fasta
Nucleotide distances: Jukes-Cantor Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Jukes-Cantor, CAT approximation with 20 rate categories
Ignored unknown character B (seen 17 times)
Ignored unknown character D (seen 5 times)
Ignored unknown character H (seen 1 times)
Ignored unknown character K (seen 115 times)
Ignored unknown character M (seen 100 times)
Ignored unknown character R (seen 181 times)
Ignored unknown character S (seen 121 times)
Ignored unknown character V (seen 13 times)
Ignored unknown character W (seen 88 times)
Ignored unknown character X (seen 1595 times)
Ignored unknown character Y (seen 190 times)
      5.44 seconds: Top hits for   1724 of   6904 seqs (at seed    100)
      8.34 seconds: Top hits fo

In [28]:
rooted_tree.rooted_tree.save("rooted_tree.qza")

'rooted_tree.qza'

In [29]:
rooted_tree = Artifact.load("rooted_tree.qza")

In [53]:
meta_table = pd.read_csv("sample_metadata.tsv",sep="\t",dtype = {"sample-id": "str"})

In [54]:
meta_table

,sample-id,sex,age_cat,refined_sugars
0,10317.000023139,female,60s,Yes
1,10317.000027811,male,40s,Yes
2,10317.000031513,female,30s,Yes
3,10317.000033280,female,60s,Yes
4,10317.000038261,female,40s,Yes
...,...,...,...,...
375,10317.000065667,female,20s,No
376,10317.000065673,female,20s,No
377,10317.000065675,female,20s,No
378,10317.000065681,female,20s,No


In [55]:
meta_table = meta_table.sort_values(by="sample-id").reset_index(drop=True)

In [56]:
otu_index = otu_table.view(pd.DataFrame).index.sort_values()

In [57]:
meta_table

,sample-id,sex,age_cat,refined_sugars
0,10317.000001166,female,40s,No
1,10317.000003100,female,40s,No
2,10317.000004936,male,50s,No
3,10317.000010862,male,20s,No
4,10317.000012018,female,50s,No
...,...,...,...,...
375,10317.000110177,male,30s,Yes
376,10317.000112912,female,50s,Yes
377,10317.000113026,female,40s,Yes
378,10317.000113088,female,30s,Yes


In [58]:
otu_index

Index(['10317.000001166.25853', '10317.000003100.69016',
       '10317.000004936.25853', '10317.000004936.30606',
       '10317.000010862.25853', '10317.000012018.56079',
       '10317.000013727.69016', '10317.000014885.41931',
       '10317.000017272.30606', '10317.000020655.26478',
       ...
       '10317.000108903.74729', '10317.000108906.74729',
       '10317.000109022.74729', '10317.000109095.74729',
       '10317.000109791.74729', '10317.000109807.76952',
       '10317.000110177.74729', '10317.000112912.76952',
       '10317.000112912.81168', '10317.000113026.81168'],
      dtype='object', length=371)

In [59]:
non_match = []
for i in range(meta_table.shape[0]):
    result = [k for k in otu_index if k.startswith(meta_table["sample-id"][i])]
    if len(result) == 0:
        non_match.append(meta_table["sample-id"][i] )
    else: 
        meta_table["sample-id"][i] = meta_table["sample-id"][i] + result[0][-6:]

In [60]:
meta_table

,sample-id,sex,age_cat,refined_sugars
0,10317.000001166.25853,female,40s,No
1,10317.000003100.69016,female,40s,No
2,10317.000004936.25853,male,50s,No
3,10317.000010862.25853,male,20s,No
4,10317.000012018.56079,female,50s,No
...,...,...,...,...
375,10317.000110177.74729,male,30s,Yes
376,10317.000112912.76952,female,50s,Yes
377,10317.000113026.81168,female,40s,Yes
378,10317.000113088,female,30s,Yes


In [61]:
meta_table_filt = meta_table.loc[~meta_table["sample-id"].isin(non_match)] 
    

In [62]:
meta_table_filt

,sample-id,sex,age_cat,refined_sugars
0,10317.000001166.25853,female,40s,No
1,10317.000003100.69016,female,40s,No
2,10317.000004936.25853,male,50s,No
3,10317.000010862.25853,male,20s,No
4,10317.000012018.56079,female,50s,No
...,...,...,...,...
373,10317.000109791.74729,male,20s,Yes
374,10317.000109807.76952,female,40s,Yes
375,10317.000110177.74729,male,30s,Yes
376,10317.000112912.76952,female,50s,Yes


In [63]:
meta_table_filt.to_csv("sample_metadata.tsv",sep="\t",index=None)

In [38]:
otu_df = otu_table.view(pd.DataFrame)

OTU filtering

In [69]:
otu_df

,2941399,4425663,353737,363646,1090458,922239,845273,185021,293665,4453060,...,213762,179905,4324040,333726,199147,1808779,173916,435264,325586,4307347
10317.000023139.25853,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000027811.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,30.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000031513.25853,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000033280.31503,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000038261.33800,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317.000065667.26478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000065673.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000065675.25853,0.0,81.0,0.0,0.0,92.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000065681.25853,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
otu_df.index.isin(meta_table_filt["sample-id"])


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,

In [75]:
otu_df = otu_df.loc[otu_df.index.isin(meta_table_filt["sample-id"])].sort_index()

In [ ]:
otu_df.transpose

In [103]:
otu_df.to_csv("otu_match.tsv", sep='\t')

In [ ]:
!biom convert -i otu_match.tsv -o otu_match.biom --table-type="OTU table" --to-hdf5

In [104]:
!qiime tools import \
  --input-path otu_match.biom\
  --type 'FeatureTable[Frequency]' \
  --input-format BIOMV210Format \
  --output-path feature_table.qza

Imported otu_match.biom as BIOMV210Format to feature_table.qza


In [113]:
!qiime feature-table transpose --i-table feature_table.qza --o-transposed-feature-table feature_table.qza

Saved FeatureTable[Frequency] to: feature_table.qza


In [114]:
from qiime2.plugins import feature_table
otu_table = Artifact.load("feature_table.qza")

In [106]:
output_viz = feature_table.visualizers.summarize(otu_table)
output_viz.visualization.save("otu_first_analysis.qzv")

'otu_first_analysis.qzv'

In [107]:
from qiime2 import Metadata
metadata = Metadata.load("sample_metadata.tsv")

In [94]:
metadata = pd.read_csv('sample_metadata.tsv', sep='\t')
metadata

,sample-id,sex,age_cat,refined_sugars
0,10317.000001166.25853,female,40s,No
1,10317.000003100.69016,female,40s,No
2,10317.000004936.25853,male,50s,No
3,10317.000010862.25853,male,20s,No
4,10317.000012018.56079,female,50s,No
...,...,...,...,...
354,10317.000109791.74729,male,20s,Yes
355,10317.000109807.76952,female,40s,Yes
356,10317.000110177.74729,male,30s,Yes
357,10317.000112912.76952,female,50s,Yes


In [89]:
metadata["sample-id"]

0      10317.000001166.25853
1      10317.000003100.69016
2      10317.000004936.25853
3      10317.000010862.25853
4      10317.000012018.56079
               ...          
354    10317.000109791.74729
355    10317.000109807.76952
356    10317.000110177.74729
357    10317.000112912.76952
358    10317.000113026.81168
Name: sample-id, Length: 359, dtype: object

In [115]:
otu_table.view(pd.DataFrame)

,2941399,4425663,353737,363646,1090458,922239,845273,185021,293665,4453060,...,213762,179905,4324040,333726,199147,1808779,173916,435264,325586,4307347
10317.000001166.25853,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000003100.69016,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000004936.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000010862.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000012018.56079,0.0,64.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317.000109791.74729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000109807.76952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000110177.74729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000112912.76952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
import biom

b = biom.load_table("otu_match.biom")
b

359 x 6904 <class 'biom.table.Table'> with 179471 nonzero entries (7% dense)

In [91]:
np.sum(otu_table.view(pd.DataFrame).columns == metadata["sample-id"])

359

In [116]:
from qiime2.plugins import diversity
core_metrics = diversity.pipelines.core_metrics_phylogenetic(table = otu_table,
                                                             phylogeny = rooted_tree,
                                                             sampling_depth = 8200,
                                                             metadata = metadata)

/opt/anaconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/opt/anaconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.3931712073149837 and the largest is 24.638091594948897.
  warn(


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/gr887277/data/c1d88312-0f0a-4593-9fd0-dddee5ee4bdd/data/feature-table.biom -t /tmp/qiime2/gr887277/data/d9692bf3-0edd-4885-936f-5cb62f3b81a8/data/tree.nwk -o /tmp/q2-AlphaDiversityFormat-sily2v3c

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

ssu -i /tmp/qiime2/gr887277/data/c1d88312-0f0a-4593-9fd0-dddee5ee4bdd/data/feature-table.biom -t /tmp/qiime2/gr887277/data/d9692bf3-0edd-4885-936f-5cb62f3b81a8/data/tree.nwk -m unweighted -o /tmp/q2-LSMatFormat-yfrd5ihu

Running external command line application. This may print messages to stdout and/or stderr.
T

/opt/anaconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.0013194195884127862 and the largest is 5.979319453635784.
  warn(
/opt/anaconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. Th

In [68]:
import pandas as pd

In [117]:
core_metrics.rarefied_table.view(pd.DataFrame)

,2941399,4425663,353737,363646,1090458,922239,845273,185021,293665,4453060,...,4356697,213762,179905,4324040,333726,199147,1808779,173916,435264,325586
10317.000001166.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000003100.69016,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000010862.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000012018.56079,0.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000013727.69016,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317.000109095.74729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000109791.74729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000109807.76952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000110177.74729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
core_metrics_rarefied_table_df = core_metrics.rarefied_table.view(pd.DataFrame).reset_index().melt(id_vars=['index'],
                                                          var_name="feature", 
                                                          value_name="Value")
core_metrics_rarefied_table_df.to_csv('core_metrics_rarefied_table_df.csv')
core_metrics_rarefied_table_df

,index,feature,Value
0,10317.000001166.25853,2941399,0.0
1,10317.000003100.69016,2941399,0.0
2,10317.000010862.25853,2941399,0.0
3,10317.000012018.56079,2941399,0.0
4,10317.000013727.69016,2941399,0.0
...,...,...,...
1785987,10317.000109095.74729,325586,0.0
1785988,10317.000109791.74729,325586,0.0
1785989,10317.000109807.76952,325586,0.0
1785990,10317.000110177.74729,325586,0.0


In [119]:
core_metrics_rarefied_table_df

,index,feature,Value
0,10317.000001166.25853,2941399,0.0
1,10317.000003100.69016,2941399,0.0
2,10317.000010862.25853,2941399,0.0
3,10317.000012018.56079,2941399,0.0
4,10317.000013727.69016,2941399,0.0
...,...,...,...
1785987,10317.000109095.74729,325586,0.0
1785988,10317.000109791.74729,325586,0.0
1785989,10317.000109807.76952,325586,0.0
1785990,10317.000110177.74729,325586,0.0


In [121]:
observed_otus_vector_group_significance = diversity.actions.alpha_group_significance(core_metrics.observed_otus_vector,
                                                                                     metadata)
observed_otus_vector_group_significance.visualization

AttributeError: 'Results' object has no attribute 'observed_otus_vector'

In [122]:
shannon_vector_group_significance = diversity.actions.alpha_group_significance(core_metrics.shannon_vector,
                                                                         metadata)
shannon_vector_group_significance.visualization.save("shannon.qzv")

'shannon.qzv'

In [123]:
faith_pd_group_significance = diversity.actions.alpha_group_significance(core_metrics.faith_pd_vector,
                                                                         metadata)
faith_pd_group_significance.visualization.save("faith_pd.qzv")

'faith_pd.qzv'

In [124]:
evenness_group_significance = diversity.actions.alpha_group_significance(core_metrics.evenness_vector,
                                                                         metadata)
evenness_group_significance.visualization.save("evenness_group.qzv")

'evenness_group.qzv'

In [125]:
bray_curtis_distance_significance = diversity.actions.beta_group_significance(core_metrics.bray_curtis_distance_matrix,
                                                                           metadata.get_column('refined_sugars'))
bray_curtis_distance_significance.visualization.save("bray_curtis.qzv")

'bray_curtis.qzv'

In [126]:
jaccard_distance_significance = diversity.actions.beta_group_significance(core_metrics.jaccard_distance_matrix,
                                                                           metadata.get_column('refined_sugars'))
jaccard_distance_significance.visualization.save("jaccard_beta.qzv")

'jaccard_beta.qzv'

In [132]:
unweighted_unifrac_significance = diversity.actions.beta_group_significance(core_metrics.unweighted_unifrac_distance_matrix,
                                                                           metadata.get_column('refined_sugars'))
unweighted_unifrac_significance.visualization.save("unweighted_unifrac.qzv")

'unweighted_unifrac.qzv'

In [131]:
weighted_unifrac_significance = diversity.actions.beta_group_significance(core_metrics.weighted_unifrac_distance_matrix,
                                                                           metadata.get_column('refined_sugars'))
weighted_unifrac_significance.visualization.save("weighted_unifrac.qzv")

'weighted_unifrac.qzv'

In [135]:
from qiime2.plugins import emperor
emperor_plot = emperor.visualizers.plot(core_metrics.unweighted_unifrac_pcoa_results,
                                        metadata)
                                        #custom_axes=['DaysSinceExperimentStart'])
emperor_plot.visualization.save('emperor.qzv')

'emperor.qzv'